## EU Dev Summit 2022

### Webhook

#### Run this cell to connect to your GIS and get started:

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

#### Get the Template Groups Content

In [ ]:
templateGroup = gis.groups.get("3281bc0b9a854323b3bd173cc282897a")

templateContent = templateGroup.content()
print(f"Found {len(templateContent)} Items in Template Group")

#### Get the WebHook belonging to the ```/groups/add``` event

In [ ]:
webhookManager = gis.admin.webhooks
createGroupWebhook = [webhook for webhook in webhookManager.list() if "/groups/add" in webhook.properties["events"]][0]
print(f"Found Webhook {createGroupWebhook.properties['name']}")

#### Get the latest WebHook Payload to retrieve GroupID

In [ ]:
import json
webhookNotifications = createGroupWebhook.notifications
if len(webhookNotifications) > 0:
    latestEvent = createGroupWebhook.notifications[0]
    createGroupEvents = [event for event in json.loads(latestEvent["payload"])["events"] if event["operation"] == "add" and event["source"] == "group"]
    newGroups = [gis.groups.get(event["id"]) for event in createGroupEvents]
    print(f"Found {len(newGroups)} newly created group(s)")
else:
    raise SystemExit("No newly created groups found")

#### Clone the Content and share to the new Group

In [ ]:
for newGroup in newGroups:
    sharedItems = [item.share(groups=[newGroup]) for item in templateContent]
    print(f"Succesfully shared {len([item for item in sharedItems if item['results'][0]['success']])} Items with group: {newGroup.title}")